# Forecasting

by Bart De Vylder and Pieter Buteneers from CoScale


## Imports

Let's first start with importing all the necessary packages. Some imports will be repeated in the exercises but if you want to skip some parts you can just execute the imports below and start with any exercise.

As you can see we also import packages from `__future__`. This is to improve the compatibility with Python 3, but will not guarantee it.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (13.0, 8.0)
%matplotlib inline

import pickle_helper

import sklearn
import sklearn.linear_model
import sklearn.preprocessing
import sklearn.gaussian_process
import sklearn.ensemble

# to make the code is compatible with python 3
from __future__ import print_function   # turns print into a function
from __future__ import division         # makes sure 3/2 = 1.5 and not 1 (use 3//2 = 1 instead)

For the forecasting we are going to use page views data, very similar to the data used in the anomaly detection section. It is also page view data and contains 1 sample per hour. 

In [ ]:
ts_data = pickle_helper.load('data/train_set_forecasting.pickle')

plt.figure(figsize=(20,4))
plt.plot(ts_data)
plt.show()

In the graph above you can clearly see that there is seasonality and a rising trend in the data.

## One-step ahead prediction

This forecasting section will describe the one-step ahead prediction. This means in this case that we will only predict the next data point which is in this case the number of pageviews in the next hour. 

As input for the prediction of point $x_{t+1}$ we use a limited history of size ```width``` ($w$) of the known points:

$$x_{t-w},\ldots  x_{t-2},  x_{t-1}, x_t \rightarrow x_{t+1} $$


In order to train a model on a timeseries we first convert in into a the training format sklearn accepts:

```
    x_train = [[x_0, x_1, x_2, ...],          y_train = [x_w,
               [x_1, x_2, x_3, ...],                     x_{w+1},  
                                ...]                          ...]

```


Let's now train a simple linear regression model on this training data.

In [ ]:
def convert_time_series_to_train_data(ts, width):
    x_train, y_train = [], []
    for i in range(len(ts) - width - 1):
##### Implement this part of the code #####
raise NotImplementedError()
#         x_train.append( ? )
#         y_train.append( ? )
    return np.array(x_train), np.array(y_train)


# some tests
width = 5
x_train, y_train = convert_time_series_to_train_data(ts_data, width)

assert x_train.shape == (303, width)
assert y_train.shape == (303,)

In [ ]:
width = 5
x_train, y_train = convert_time_series_to_train_data(ts_data, width)
model = sklearn.linear_model.LinearRegression()
model.fit(x_train, y_train)
print('The R2 score of the linear model with width =', width, 'is', model.score(x_train, y_train))

Now we're going to visualize our prediction by repeating the one-step-ahead prediction. 

In [ ]:
# this is a helper function to make the predictions
def predict(model, ts_data, width, nof_points):
    prediction = []
    # create the input data set for the first predicted output
    # copy the data to make sure the original is not overwritten
    x_test = np.copy(ts_data[-width : ])
    for i in range(nof_points):
        # predict only the next data point
        prediction.append(model.predict(x_test.reshape((1, -1)))[0])
        # use the newly predicted data point as input for the next prediction
        x_test[0 : -1] = x_test[1 : ]
##### Implement this part of the code #####
raise NotImplementedError()
        # x_test[-1] = ?

    return np.array(prediction)

In [ ]:
nof_predictions = 200
prediction = predict(model, ts_data, width, nof_predictions)

plt.figure(figsize=(20,4))
plt.plot(np.concatenate((ts_data, prediction)), 'g')
plt.plot(ts_data, 'b')
plt.show()

As you can see in the image above the prediction is not very good. Let's play a bit with the ```width``` variable below to see if you can find a better forecast. 

In [ ]:
##### Implement this part of the code #####
raise NotImplementedError()
# width = ?

x_train, y_train = convert_time_series_to_train_data(ts_data, width)
model = sklearn.linear_model.LinearRegression()
model.fit(x_train, y_train)
print('The R2 score of the linear model with width =', width, 'is', model.score(x_train, y_train))

prediction = predict(model, ts_data, width, 200)

plt.figure(figsize=(20,4))
plt.plot(np.concatenate((ts_data, prediction)), 'g')
plt.plot(ts_data, 'b')
plt.show()

assert width > 1

If everything went well you found that in this case the prediction task performs better if the model receives more data. In particular, due to the seasonality the value of the day before is an important part of the prediction. This we can also verify by inspecting the coefficients assigned by the linear method: 

In [ ]:
print(model.coef_)

plt.figure(figsize=(8,4))
plt.bar(range(-width+1, 1),  model.coef_)
plt.xlabel("lag")
plt.ylabel("coefficient for prediction")
plt.show()

Now try the same thing for the following models:
* ```sklearn.linear_model.RidgeCV()```
* ```sklearn.linear_model.LassoCV()```

Both models also estimate the noise that is present in the data to avoid overfitting. `RidgeCV()` will keep the weights that are found small, but it won't put them to zero. `LassoCV()` on the other hand will put several weights to 0 (inspect the ```model.coef_``` to verify). 

## Automation

What we have done up to now is manually selecting the best outcome based on the test result. This can be considered cheating because you have just created a self-fulfilling prophecy. Additionally it is not only cheating it is also hard to find the exact `width` that gives the best result by just visually inspecting it. So we need a more objective approach to solve this.

To automate this process you can use a validation set. In this case we will use the last 48 hours of the training set to validate the score and select the best parameter value. This means that we will have to use a subset of the training set to fit the model.

In [ ]:
model_generators = [sklearn.linear_model.LinearRegression, sklearn.linear_model.RidgeCV,
                    sklearn.linear_model.LassoCV]
best_score = 0

##### Implement this part of the code #####
raise NotImplementedError()
# for model_gen in ? :
#    for width in range( ? , ? ): 
        x_train, y_train = convert_time_series_to_train_data(ts_data, width)
        # train the model on the first 48 hours
        x_train_i, y_train_i = x_train[ : -48, :], y_train[ : -48]
        # use the last 48 hours for validation
        x_val_i, y_val_i = x_train[-48 : ], y_train[-48 : ]
        
        ##### Implement this part of the code #####
        raise NotImplementedError()
        # model = 
        
        # there is a try except clause here because some models do not converge for some data
        try:
            ##### Implement this part of the code #####
            raise NotImplementedError()
            # model.fit( ? , ? )
            # this_score = ?
            
            if this_score > best_score:
                best_score = this_score
                best_model_gen = model_gen
                best_width = width
        except:
            pass

print(best_model_gen().__class__, 'was selected as the best model with a width of', best_width,
      'and a validation R2 score of', best_score)

If everything is correct the LassoCV methods was selected.

Now we are going to train this best model on all the data. In this way we use all the available data to build a model.

In [ ]:
##### Implement this part of the code #####
raise NotImplementedError()
# width = ?
# model = ?

x_train, y_train = convert_time_series_to_train_data(ts_data, width)

##### Implement this part of the code #####
raise NotImplementedError()
# model.fit( ? , ? )

nof_predictions = 200
prediction = predict(model, ts_data, width, nof_predictions)

plt.figure(figsize=(20,4))
plt.plot(np.concatenate((ts_data, prediction)), 'g')
plt.plot(ts_data, 'b')
plt.show()

Altough the optimal result found here might not be the best visually, it is a far better result than the one you selected manually just because there was no cheating involved ;-).

Some additional info:
* This noise level of `RidgeCV()` and `LassoCV()` is estimated by automatically performing train and validation within the method itself. This will make them much more robust against over-fitting. The actual method used is [Cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) which is a better approach of what we do here because it repeats the training and validation multiple times for different training and validation sets. The parameter that is set for these methods is often called the regularization parameter in literature and is well suited to avoid over-fitting.